In [1]:
# ติดตั้ง mediapipe แบบเงียบๆ (-q)
!pip install mediapipe opencv-python tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 39.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.8 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
google-spark-connect 0.5.2 requires google-api-core>=2.19.1, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-bigtable 2.30.0 requires google-api-core[grpc]<3.0.0,>=2.16.0, but you have google

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# from mediapipe.tasks.python.core.base_options import BaseOptions, Delegate
from tqdm.auto import tqdm # <-- 1. เพิ่มการ import tqdm.auto
import pandas as pd

2025-07-05 13:53:41.395459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751723621.619559      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751723621.691490      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
MODEL_PATH = '/kaggle/input/mediapipe/face_landmarker.task'

BASE_INPUT_PATH = '/kaggle/input/ferplus/FER2013' 

BASE_OUTPUT_PATH = '/kaggle/working/output'

In [4]:
csv_path = "/kaggle/input/ferplus/fer2013new.csv"
df = pd.read_csv(csv_path)

# กรอง: ไม่เอา contempt กับ NF
df = df[(df['contempt'] == 0) & (df['NF'] == 0)].copy()

# อารมณ์ 7 แบบที่เราจะใช้
emotion_cols = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear']
df['label'] = df[emotion_cols].idxmax(axis=1)

# จัดชื่อคอลัมน์และ split
df = df.rename(columns={'Image name': 'filename', 'Usage': 'split'})
df['filename'] = df['filename'].str.strip()
df['split'] = df['split'].replace({
    'Training': 'train',
    'PublicTest': 'valid',
    'PrivateTest': 'test'
})

# เก็บเฉพาะคอลัมน์ที่ต้องใช้
df_final = df[['filename', 'label', 'split']]

# บันทึกลงไฟล์ใน Kaggle Working Directory
df_final.to_csv("/kaggle/working/index_with_labels.csv", index=False)

# ตรวจสอบ
df_final.head()

,filename,label,split
0,fer0000000.png,neutral,train
1,fer0000001.png,neutral,train
2,fer0000002.png,neutral,train
3,fer0000003.png,neutral,train
4,fer0000004.png,neutral,train


In [5]:
IMPORTANT_LANDMARK_INDICES = {
    # ริมฝีปาก (Lips)
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308,
    95, 88, 178, 87, 14, 317, 402, 318, 324,
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291,
    146, 91, 181, 84, 17, 314, 405, 321, 375,
    # ตาซ้าย (Left Eye)
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
    # คิ้วซ้าย (Left Eyebrow)
    70, 63, 105, 66, 107, 55, 65, 52, 53, 46,
    # ตาขวา (Right Eye)
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
    # คิ้วขวา (Right Eyebrow)
    300, 293, 334, 296, 336, 285, 295, 282, 283, 276
}

In [6]:
def create_landmarker():
    base_options = python.BaseOptions(model_asset_path=MODEL_PATH)
    options = vision.FaceLandmarkerOptions(base_options=base_options, running_mode=vision.RunningMode.IMAGE, num_faces=1)
    print("สร้าง Landmarker สำหรับ CPU สำเร็จ")
    return vision.FaceLandmarker.create_from_options(options)

In [7]:
def extract_landmarks_data(rgb_image, detection_result):
    extracted_faces_data = []
    if not detection_result.face_landmarks:
        return extracted_faces_data

    height, width, _ = rgb_image.shape

    for face_landmarks in detection_result.face_landmarks:
        # Store landmarks as a NumPy array (N, 3) for x, y, and z
        important_landmarks_array = []
        important_landmarks_indices_list = []

        for i, landmark in enumerate(face_landmarks):
            if i in IMPORTANT_LANDMARK_INDICES:
                # 1. ดึงค่า z เพิ่มเข้ามา
                # ค่า z ของ MediaPipe บอกความลึก โดยมีจุดศูนย์กลางของศีรษะเป็น origin
                # เราจะคูณด้วย width เพื่อให้สเกลใกล้เคียงกับ x และ y
                x = int(landmark.x * width)
                y = int(landmark.y * height)
                z = int(landmark.z * width) # ดึงค่า z และแปลงเป็นพิกัด

                # (สำคัญ) เพิ่ม z เข้าไปใน array
                important_landmarks_array.append([x, y, z])
                important_landmarks_indices_list.append(i)

        # Convert to numpy array, shape will now be (N, 3)
        important_landmarks_array = np.array(important_landmarks_array, dtype=np.int32)
        important_landmarks_indices_array = np.array(important_landmarks_indices_list, dtype=np.int32)

        # Store connections as a NumPy array (M, 6)
        # Each row: [start_x, start_y, start_z, end_x, end_y, end_z]
        important_connections_array = []
        
        # Create a quick lookup for landmark coordinates by index
        landmark_coords_map = {idx: coord for idx, coord in zip(important_landmarks_indices_list, important_landmarks_array)}

        connections = mp.solutions.face_mesh.FACEMESH_TESSELATION
        for connection in connections:
            start_idx, end_idx = connection

            if start_idx in IMPORTANT_LANDMARK_INDICES and end_idx in IMPORTANT_LANDMARK_INDICES:
                if start_idx in landmark_coords_map and end_idx in landmark_coords_map:
                    # 2. แก้ไขการดึงข้อมูลจาก map ให้รองรับ z ( unpacking 3 values)
                    start_x, start_y, start_z = landmark_coords_map[start_idx]
                    end_x, end_y, end_z = landmark_coords_map[end_idx]
                    
                    # 3. เพิ่มพิกัด z ของจุดเริ่มต้นและสิ้นสุดเข้าไปใน array
                    important_connections_array.append([start_x, start_y, start_z, end_x, end_y, end_z])
        
        important_connections_array = np.array(important_connections_array, dtype=np.int32)

        extracted_faces_data.append({
            'important_landmarks_coords': important_landmarks_array,
            'important_landmarks_indices': important_landmarks_indices_array,
            'important_connections_coords': important_connections_array
        })
    return extracted_faces_data

In [11]:
def process_images_in_folder(landmarker, dataframe, base_input_path, output_filename="all_data_with_landmarks.npz"):
    all_landmarks = []
    all_landmark_indices = []
    all_connections = []
    all_labels = []
    all_splits = []
    all_filenames = []

    print(f"\n--- เริ่มประมวลผลรูปภาพทั้งหมดและรวมเป็นไฟล์เดียว ---")
    # Iterate through each row of the DataFrame
    for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Processing images"):
        filename = row['filename']
        label = row['label']
        split = row['split']

        # Determine the correct subfolder based on the 'split' column
        if split.lower() == 'train':
            input_folder = os.path.join(base_input_path, 'FER2013TRAIN')
        elif split.lower() == 'test':
            input_folder = os.path.join(base_input_path, 'FER2013TEST')
        elif split.lower() == 'valid': # Assuming 'valid' for validation set
            input_folder = os.path.join(base_input_path, 'FER2013VALID')
        else:
            print(f"คำเตือน: ไม่รู้จัก split type '{split}' สำหรับไฟล์ '{filename}'. ข้าม.")
            continue

        input_image_path = os.path.join(input_folder, filename)

        if not os.path.exists(input_image_path):
            print(f"คำเตือน: ไม่พบไฟล์รูปภาพที่: {input_image_path}. ข้าม.")
            continue

        image_bgr = cv2.imread(input_image_path)
        if image_bgr is None:
            print(f"คำเตือน: ไม่สามารถอ่านรูปภาพได้: {input_image_path}. ข้าม.")
            continue
        
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
        detection_result = landmarker.detect(mp_image)
        
        extracted_data = extract_landmarks_data(image_rgb, detection_result)
        
        if extracted_data: # Only process if a face was detected
            face_data = extracted_data[0] # Assuming one face per image
            all_landmarks.append(face_data['important_landmarks_coords'])
            all_landmark_indices.append(face_data['important_landmarks_indices'])
            all_connections.append(face_data['important_connections_coords'])
            all_labels.append(label)
            all_splits.append(split)
            all_filenames.append(filename)
        # else:
            # print(f"ไม่พบใบหน้าใน {filename}, ข้อมูล landmark จะไม่ถูกบันทึกสำหรับรูปภาพนี้")

    # Create output directory if it doesn't exist
    if not os.path.exists(BASE_OUTPUT_PATH):
        os.makedirs(BASE_OUTPUT_PATH)
        print(f"สร้างโฟลเดอร์ผลลัพธ์: {BASE_OUTPUT_PATH}")

    output_filepath = os.path.join(BASE_OUTPUT_PATH, output_filename)
    
    # Save all collected data into a single .npz file
    np.savez_compressed(output_filepath,
                       landmarks=np.array(all_landmarks, dtype=object), # Use dtype=object for variable-length arrays
                       landmark_indices=np.array(all_landmark_indices, dtype=object),
                       connections=np.array(all_connections, dtype=object),
                       labels=np.array(all_labels),
                       splits=np.array(all_splits),
                       filenames=np.array(all_filenames))

    print(f"\nบันทึกข้อมูล Landmarker, label, และ split ทั้งหมดลงในไฟล์: '{output_filepath}' สำเร็จ")

In [12]:
def main():
    if not os.path.exists(MODEL_PATH):
        print(f"!!! ไม่พบไฟล์โมเดลที่: '{MODEL_PATH}'")
        print("โปรดดาวน์โหลดโมเดล Face Landmarker จาก MediaPipe และระบุ MODEL_PATH ให้ถูกต้อง")
        print("ตัวอย่าง: https://developers.google.com/mediapipe/solutions/vision/face_landmarker#models")
        return
    if not os.path.exists(BASE_INPUT_PATH):
        print(f"!!! ไม่พบโฟลเดอร์ข้อมูลที่: '{BASE_INPUT_PATH}'")
        print("โปรดตรวจสอบ BASE_INPUT_PATH ให้ถูกต้อง (e.g., 'data/FER2013')")
        return
    landmarker = create_landmarker()
    
    # Process all images and save to a single .npz file
    process_images_in_folder(landmarker, df_final, BASE_INPUT_PATH)

    landmarker.close()
    print(f"\nการประมวลผลทั้งหมดเสร็จสิ้น! ตรวจสอบไฟล์ข้อมูลพิกัด (ในรูปแบบ .npz) ได้ที่โฟลเดอร์ '{BASE_OUTPUT_PATH}'")

In [13]:
if __name__ == '__main__':
    main()

สร้าง Landmarker สำหรับ CPU สำเร็จ

--- เริ่มประมวลผลรูปภาพทั้งหมดและรวมเป็นไฟล์เดียว ---


I0000 00:00:1751724058.632768      35 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:84) egl_initializedUnable to initialize EGL
W0000 00:00:1751724058.633811      35 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1751724058.644765     106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751724058.683665     107 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processing images:   0%|          | 0/31546 [00:00<?, ?it/s]

สร้างโฟลเดอร์ผลลัพธ์: /kaggle/working/output

บันทึกข้อมูล Landmarker, label, และ split ทั้งหมดลงในไฟล์: '/kaggle/working/output/all_data_with_landmarks.npz' สำเร็จ

การประมวลผลทั้งหมดเสร็จสิ้น! ตรวจสอบไฟล์ข้อมูลพิกัด (ในรูปแบบ .npz) ได้ที่โฟลเดอร์ '/kaggle/working/output'
